# Meteorological stations coverage extraction


Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is part of the EStreams publication and was used to extract and aggregate the weather stations coverage information from the E-OBS dataset.

* Note that this code enables not only the replicability of the current database but also the extrapolation to new catchment areas. 
* Additionally, the user should download and insert the original raw-data in the folder of the same name prior to run this code. 
* The original third-party data used were not made avaialable in this repository due to redistribution and storage-space reasons.  

## Requirements
**Python:**

* Python>=3.6
* Jupyter
* geopandas=0.10.2
* numpy
* os
* pandas
* shapely
* tqdm

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**

* data/shapefiles/estreams_catchments.shp
* data/eobs_stations/stations_info_{rr, tg, tn, tx, pp, hu, fg, qq}_v28.0e.txt. https://www.ecad.eu/download/ensembles/download.php (Last access: 27 November 2023)

**Directory:**

* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 

## References
* Cornes, R., G. van der Schrier, E.J.M. van den Besselaar, and P.D. Jones. 2018: An Ensemble Version of the E-OBS Temperature and Precipitation Datasets, J. Geophys. Res. Atmos., 123. doi:10.1029/2017JD028200

## Licenses
* EOBS: "The ECA&D data policy applies. These observational data are strictly for use in non-commercial research and non-commercial education projects only. Scientific results based on these data must be submitted for publication in the open literature without any delay linked to commercial objectives" https://www.ecad.eu/download/ensembles/download.php#guidance (Last access: 27 November 2023)

# Import modules

In [17]:
import geopandas as gpd
import os
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon
import tqdm as tqdm
from utils.hydrology import count_geometries_in_polygons

# Configurations

In [3]:
# Only editable variables:
# Relative path to your local directory
PATH = "../../.."
# Do not change the order of the variables:
filenames = ['data/eobs_stations/stations_info_qq_v28.0e.txt', 'data/eobs_stations/stations_info_pp_v28.0e.txt',
             'data/eobs_stations/stations_info_tg_v28.0e.txt','data/eobs_stations/stations_info_tx_v28.0e.txt',
             'data/eobs_stations/stations_info_fg_v28.0e.txt','data/eobs_stations/stations_info_rr_v28.0e.txt',
             'data/eobs_stations/stations_info_tn_v28.0e.txt','data/eobs_stations/stations_info_hu_v28.0e.txt']

* #### The users should NOT change anything in the code below here.


In [4]:
# Non-editable variables:
PATH_OUTPUT = "results/staticattributes/"
# Set the directory:
os.chdir(PATH)

# Import data
## Catchment boundaries

In [5]:
catchment_boundaries = gpd.read_file('data/shapefiles/estreams_catchments.shp')
catchment_boundaries

,id,area_km2,outlet_lat,outlet_lng,name,area_offic,layer,path,area_diff,area_calc,basin_id,geometry
0,HUGR020,"9,600",46.785,21.142,6444410,9011,HUGR020,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,6.536,9595.794,HUGR020,"POLYGON ((21.13208 46.77291, 21.13208 46.77375..."
1,HUGR021,"189,000",46.423,18.896,6442080,189538,HUGR021,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.284,188597.110,HUGR021,"POLYGON ((18.91708 46.41791, 18.91708 46.41625..."
2,HUGR022,"28,500",48.126,22.340,6444304,29057,HUGR022,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-1.917,28507.473,HUGR022,"POLYGON ((22.32875 48.10875, 22.32791 48.10875..."
3,HUGR023,"188,000",46.627,18.869,6442060,189092,HUGR023,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.577,188286.167,HUGR023,"POLYGON ((18.89041 46.62875, 18.88875 46.62708..."
4,HUGR025,"1,210",47.662,19.683,6444240,1222,HUGR025,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.982,1206.441,HUGR025,"POLYGON ((19.68124 47.66875, 19.68291 47.66875..."
5,HUGR026,110,46.891,20.498,6444420,26647,HUGR026,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-99.587,109.639,HUGR026,"POLYGON ((20.49958 46.93125, 20.49958 46.93125..."
6,HUGR027,"4,490",48.497,21.229,6444330,4515,HUGR027,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.554,4494.402,HUGR027,"POLYGON ((21.23458 48.49708, 21.23208 48.49708..."
7,HUGR028,"5,770",46.883,18.141,6442110,5884,HUGR028,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-1.937,5773.506,HUGR028,"POLYGON ((16.56458 46.93291, 16.56541 46.93291..."
8,HUGR029,"185,000",47.495,19.048,6442040,184893,HUGR029,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,0.058,184810.677,HUGR029,"POLYGON ((19.11291 47.48291, 19.11125 47.48458..."
9,HUGR030,"13,000",46.419,16.695,6446100,13033,HUGR030,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.253,13044.665,HUGR030,"POLYGON ((16.63625 46.45625, 16.63541 46.45625..."


In [6]:
print("The total number of catchments to be processed are:", len(catchment_boundaries))

The total number of catchments to be processed are: 33


In [7]:
# Here you can check the crs of the datasets:
print("CRS of catchment_boundaries:", catchment_boundaries.crs)

CRS of catchment_boundaries: epsg:4326


In [8]:
# Define the target CRS to ETRS89 LAEA (3035)
target_crs = 'EPSG:3035'  

# Reproject the GeoDataFrame to the target CRS
catchment_boundaries_reprojected = catchment_boundaries.to_crs(target_crs)

### E-OBS stations

In [9]:
# Here we analyse the precipitation gauges:
filename = filenames[5]
filename

'data/eobs_stations/stations_info_rr_v28.0e.txt'

In [10]:
# Use read_csv with the '|' delimiter
eobs_stations = pd.read_csv('data/eobs_stations/stations_info_rr_v28.0e.txt', delimiter='|', encoding='latin1')
eobs_stations.columns = ['STATION', 'NAME','COUNTRY', 'LAT', 'LON', 'ELEV',
       'START', 'STOP']
eobs_stations

,STATION,NAME,COUNTRY,LAT,LON,ELEV,START,STOP
0,1,Vaexjoe,SWEDEN,56.87,14.80,166.0,1950-01-01,2019-12-31
1,2,Falun,SWEDEN,60.62,15.62,160.0,1950-01-01,2023-06-30
2,3,Stensele,SWEDEN,65.07,17.15,325.0,1950-01-01,2004-12-31
3,4,Linkoeping,SWEDEN,58.40,15.53,93.0,1950-01-01,2023-06-30
4,5,Linkoeping-Malmslaett,SWEDEN,58.40,15.53,93.0,1950-01-01,2023-06-30
...,...,...,...,...,...,...,...,...
18689,26309,KLOEFTA,NORWAY,60.07,11.13,162.0,1956-01-01,2023-06-30
18690,26310,JESSHEIM,NORWAY,60.15,11.18,203.0,1956-01-01,2023-06-30
18691,26317,EIKELANDSOSEN,NORWAY,60.25,5.74,82.0,2023-04-01,2023-06-30
18692,26318,MIDTBOTN,NORWAY,59.92,6.18,827.0,2023-04-01,2023-06-30


In [23]:
# Create your dataframes
eobs_stations_qq = pd.DataFrame()
eobs_stations_pp = pd.DataFrame()
eobs_stations_tg = pd.DataFrame()
eobs_stations_tx = pd.DataFrame()
eobs_stations_fg = pd.DataFrame()
eobs_stations_rr = pd.DataFrame()
eobs_stations_tn = pd.DataFrame()
eobs_stations_hu = pd.DataFrame()

# Store dataframes in a dictionary
dataframes = {
    'qq': eobs_stations_qq,
    'pp': eobs_stations_pp,
    'tg': eobs_stations_tg,
    'tx': eobs_stations_tx,
    'fg': eobs_stations_fg,
    'rr': eobs_stations_rr,
    'tn': eobs_stations_tn,
    'hu': eobs_stations_hu
}

selected_variables = ['qq', 'pp', 'tg', 'tx', 'fg', 'rr', 'tn', 'hu']

i = 0

for filename in filenames:
    # Use read_csv with the '|' delimiter
    eobs_stations = pd.read_csv(filename, delimiter='|', encoding='latin1')
    eobs_stations.columns = ['STATION', 'NAME','COUNTRY', 'LAT', 'LON', 'ELEV',
       'START', 'STOP']
    
    dataframes[selected_variables[i]] = eobs_stations
    
    i = i + 1

In [24]:
dataframes[selected_variables[5]]

,STATION,NAME,COUNTRY,LAT,LON,ELEV,START,STOP
0,1,Vaexjoe,SWEDEN,56.87,14.80,166.0,1950-01-01,2019-12-31
1,2,Falun,SWEDEN,60.62,15.62,160.0,1950-01-01,2023-06-30
2,3,Stensele,SWEDEN,65.07,17.15,325.0,1950-01-01,2004-12-31
3,4,Linkoeping,SWEDEN,58.40,15.53,93.0,1950-01-01,2023-06-30
4,5,Linkoeping-Malmslaett,SWEDEN,58.40,15.53,93.0,1950-01-01,2023-06-30
...,...,...,...,...,...,...,...,...
18689,26309,KLOEFTA,NORWAY,60.07,11.13,162.0,1956-01-01,2023-06-30
18690,26310,JESSHEIM,NORWAY,60.15,11.18,203.0,1956-01-01,2023-06-30
18691,26317,EIKELANDSOSEN,NORWAY,60.25,5.74,82.0,2023-04-01,2023-06-30
18692,26318,MIDTBOTN,NORWAY,59.92,6.18,827.0,2023-04-01,2023-06-30


In [25]:
# Here we convert the dataframes to geodataframes and set the projected system
# We need to reproject our geodataframes to a projected coordinate system (in meters) in order to 
# provide the buffer correctly.

for variable in selected_variables:
    # Convert the DataFrame to a GeoDataFrame
    geometry = [Point(lon, lat) for lon, lat in zip(dataframes[variable]['LON'], dataframes[variable]['LAT'])]
    dataframes[variable] = gpd.GeoDataFrame(dataframes[variable], geometry=geometry)

    # Set the coordinate reference system (CRS) for WGS-84
    dataframes[variable].crs = 'EPSG:4326'
    
    # Define the target CRS to ETRS89 LAEA (3035)
    target_crs = 'EPSG:3035'  

    # Reproject the GeoDataFrame to the target CRS
    dataframes[variable] = dataframes[variable].to_crs(target_crs)

## Buffer of the catchments boundaries
* This may take several minutes to run

In [26]:
subset_catchment = catchment_boundaries_reprojected.copy()

# First we make a buffer of 10 km around the catchment shapefiles 
buffer_distance = 10000
buffered_catchment_boundaries_reprojected = subset_catchment.copy()
buffered_catchment_boundaries_reprojected['geometry'] = subset_catchment['geometry'].buffer(buffer_distance)

## Computation

In [27]:
selected_variables = ['qq', 'pp', 'tg', 'tx', 'fg', 'rr', 'tn', 'hu']

# First we create an empty dataframe:
num_stations = pd.DataFrame()
num_stations["area"] = buffered_catchment_boundaries_reprojected.set_index("basin_id", inplace = False).area_calc

for variable in tqdm.tqdm(selected_variables):
    
    # Here we use utils.hydrology.count_geometries_in_polygons function
    num_stations["stations_num_"+variable] = count_geometries_in_polygons(dataframes[variable], 
                                                                 buffered_catchment_boundaries_reprojected, "basin_id", 
                                                                 new_column="num")
    
    num_stations["stations_dens_"+variable] = num_stations["stations_num_"+variable] / num_stations["area"]

100%|████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00,  8.17it/s]


In [28]:
num_stations

,area,stations_num_qq,stations_dens_qq,stations_num_pp,stations_dens_pp,stations_num_tg,stations_dens_tg,stations_num_tx,stations_dens_tx,stations_num_fg,stations_dens_fg,stations_num_rr,stations_dens_rr,stations_num_tn,stations_dens_tn,stations_num_hu,stations_dens_hu
basin_id,,,,,,,,,,,,,,,,,
HUGR020,9595.794,1.0,0.000104,0.0,0.000000,0.0,0.000000,1.0,0.000104,12.0,0.001251,1.0,0.000104,1.0,0.000104,0.0,0.000000
HUGR021,188597.110,508.0,0.002694,433.0,0.002296,856.0,0.004539,866.0,0.004592,693.0,0.003674,1810.0,0.009597,871.0,0.004618,833.0,0.004417
HUGR022,28507.473,1.0,0.000035,0.0,0.000000,3.0,0.000105,5.0,0.000175,21.0,0.000737,12.0,0.000421,5.0,0.000175,0.0,0.000000
HUGR023,188286.167,508.0,0.002698,433.0,0.002300,856.0,0.004546,866.0,0.004599,693.0,0.003681,1810.0,0.009613,871.0,0.004626,833.0,0.004424
HUGR025,1206.441,1.0,0.000829,0.0,0.000000,0.0,0.000000,1.0,0.000829,0.0,0.000000,1.0,0.000829,1.0,0.000829,0.0,0.000000
HUGR026,109.639,1.0,0.009121,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
HUGR027,4494.402,3.0,0.000667,2.0,0.000445,2.0,0.000445,2.0,0.000445,2.0,0.000445,3.0,0.000667,2.0,0.000445,2.0,0.000445
HUGR028,5773.506,4.0,0.000693,0.0,0.000000,1.0,0.000173,3.0,0.000520,8.0,0.001386,3.0,0.000520,3.0,0.000520,1.0,0.000173
HUGR029,184810.677,508.0,0.002749,433.0,0.002343,856.0,0.004632,866.0,0.004686,691.0,0.003739,1810.0,0.009794,871.0,0.004713,833.0,0.004507


In [29]:
num_stations_coverage = num_stations.iloc[:, 1:]
num_stations_coverage

,stations_num_qq,stations_dens_qq,stations_num_pp,stations_dens_pp,stations_num_tg,stations_dens_tg,stations_num_tx,stations_dens_tx,stations_num_fg,stations_dens_fg,stations_num_rr,stations_dens_rr,stations_num_tn,stations_dens_tn,stations_num_hu,stations_dens_hu
basin_id,,,,,,,,,,,,,,,,
HUGR020,1.0,0.000104,0.0,0.000000,0.0,0.000000,1.0,0.000104,12.0,0.001251,1.0,0.000104,1.0,0.000104,0.0,0.000000
HUGR021,508.0,0.002694,433.0,0.002296,856.0,0.004539,866.0,0.004592,693.0,0.003674,1810.0,0.009597,871.0,0.004618,833.0,0.004417
HUGR022,1.0,0.000035,0.0,0.000000,3.0,0.000105,5.0,0.000175,21.0,0.000737,12.0,0.000421,5.0,0.000175,0.0,0.000000
HUGR023,508.0,0.002698,433.0,0.002300,856.0,0.004546,866.0,0.004599,693.0,0.003681,1810.0,0.009613,871.0,0.004626,833.0,0.004424
HUGR025,1.0,0.000829,0.0,0.000000,0.0,0.000000,1.0,0.000829,0.0,0.000000,1.0,0.000829,1.0,0.000829,0.0,0.000000
HUGR026,1.0,0.009121,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
HUGR027,3.0,0.000667,2.0,0.000445,2.0,0.000445,2.0,0.000445,2.0,0.000445,3.0,0.000667,2.0,0.000445,2.0,0.000445
HUGR028,4.0,0.000693,0.0,0.000000,1.0,0.000173,3.0,0.000520,8.0,0.001386,3.0,0.000520,3.0,0.000520,1.0,0.000173
HUGR029,508.0,0.002749,433.0,0.002343,856.0,0.004632,866.0,0.004686,691.0,0.003739,1810.0,0.009794,871.0,0.004713,833.0,0.004507


In [30]:
num_stations_coverage.columns

Index(['stations_num_qq', 'stations_dens_qq', 'stations_num_pp',
       'stations_dens_pp', 'stations_num_tg', 'stations_dens_tg',
       'stations_num_tx', 'stations_dens_tx', 'stations_num_fg',
       'stations_dens_fg', 'stations_num_rr', 'stations_dens_rr',
       'stations_num_tn', 'stations_dens_tn', 'stations_num_hu',
       'stations_dens_hu'],
      dtype='object')

In [31]:
# List of values for replacement
old_values = ['_qq', '_pp', '_tg', '_tx', '_fg', '_rr', '_tn', '_hu']
new_values = ['_swr', '_sp', '_tmean', '_tmax', '_ws', '_p', '_tmin', '_rh']

# Create a mapping dictionary
column_name_mapping = {old: new for old, new in zip(old_values, new_values)}

# Replace the specified patterns in column names
num_stations_coverage.columns = num_stations_coverage.columns.to_series().replace(column_name_mapping, regex=True).values
num_stations_coverage

,stations_num_swr,stations_dens_swr,stations_num_sp,stations_dens_sp,stations_num_tmean,stations_dens_tmean,stations_num_tmax,stations_dens_tmax,stations_num_ws,stations_dens_ws,stations_num_p,stations_dens_p,stations_num_tmin,stations_dens_tmin,stations_num_rh,stations_dens_rh
basin_id,,,,,,,,,,,,,,,,
HUGR020,1.0,0.000104,0.0,0.000000,0.0,0.000000,1.0,0.000104,12.0,0.001251,1.0,0.000104,1.0,0.000104,0.0,0.000000
HUGR021,508.0,0.002694,433.0,0.002296,856.0,0.004539,866.0,0.004592,693.0,0.003674,1810.0,0.009597,871.0,0.004618,833.0,0.004417
HUGR022,1.0,0.000035,0.0,0.000000,3.0,0.000105,5.0,0.000175,21.0,0.000737,12.0,0.000421,5.0,0.000175,0.0,0.000000
HUGR023,508.0,0.002698,433.0,0.002300,856.0,0.004546,866.0,0.004599,693.0,0.003681,1810.0,0.009613,871.0,0.004626,833.0,0.004424
HUGR025,1.0,0.000829,0.0,0.000000,0.0,0.000000,1.0,0.000829,0.0,0.000000,1.0,0.000829,1.0,0.000829,0.0,0.000000
HUGR026,1.0,0.009121,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
HUGR027,3.0,0.000667,2.0,0.000445,2.0,0.000445,2.0,0.000445,2.0,0.000445,3.0,0.000667,2.0,0.000445,2.0,0.000445
HUGR028,4.0,0.000693,0.0,0.000000,1.0,0.000173,3.0,0.000520,8.0,0.001386,3.0,0.000520,3.0,0.000520,1.0,0.000173
HUGR029,508.0,0.002749,433.0,0.002343,856.0,0.004632,866.0,0.004686,691.0,0.003739,1810.0,0.009794,871.0,0.004713,833.0,0.004507


In [32]:
# Here we sort the index:
num_stations_coverage = num_stations_coverage.sort_index(axis=0)
num_stations_coverage

,stations_num_swr,stations_dens_swr,stations_num_sp,stations_dens_sp,stations_num_tmean,stations_dens_tmean,stations_num_tmax,stations_dens_tmax,stations_num_ws,stations_dens_ws,stations_num_p,stations_dens_p,stations_num_tmin,stations_dens_tmin,stations_num_rh,stations_dens_rh
basin_id,,,,,,,,,,,,,,,,
HUGR019,8.0,0.000128,3.0,0.000048,14.0,0.000224,18.0,0.000288,39.0,0.000624,42.0,0.000672,18.0,0.000288,10.0,0.000160
HUGR020,1.0,0.000104,0.0,0.000000,0.0,0.000000,1.0,0.000104,12.0,0.001251,1.0,0.000104,1.0,0.000104,0.0,0.000000
HUGR021,508.0,0.002694,433.0,0.002296,856.0,0.004539,866.0,0.004592,693.0,0.003674,1810.0,0.009597,871.0,0.004618,833.0,0.004417
HUGR022,1.0,0.000035,0.0,0.000000,3.0,0.000105,5.0,0.000175,21.0,0.000737,12.0,0.000421,5.0,0.000175,0.0,0.000000
HUGR023,508.0,0.002698,433.0,0.002300,856.0,0.004546,866.0,0.004599,693.0,0.003681,1810.0,0.009613,871.0,0.004626,833.0,0.004424
HUGR024,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.0,0.000308,0.0,0.000000,0.0,0.000000,0.0,0.000000
HUGR025,1.0,0.000829,0.0,0.000000,0.0,0.000000,1.0,0.000829,0.0,0.000000,1.0,0.000829,1.0,0.000829,0.0,0.000000
HUGR026,1.0,0.009121,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
HUGR027,3.0,0.000667,2.0,0.000445,2.0,0.000445,2.0,0.000445,2.0,0.000445,3.0,0.000667,2.0,0.000445,2.0,0.000445


In [ ]:
# Round the data to 3 decimals
num_stations_coverage = num_stations_coverage.astype(float).round(3)
num_stations_coverage

## Data export

In [22]:
# Export the final analysis:
num_stations_coverage.to_csv(PATH_OUTPUT+"/estreams_meteorology_density.csv")

# End